In [1]:
#!pip install pytorch-lightning timm optuna imagehash

In [11]:
import numpy as np
import pandas as pd
import torch
import optuna
from optuna.integration import PyTorchLightningPruningCallback
import pytorch_lightning as pl
from sklearn.metrics import mean_squared_error
import os
import shutil

from pet_finder_library import PetDataMining, PetDataLoader, PetFinderTransferModelBCEWithAddFeatures


INPUT_DATA_PREFIX = '.'

In [5]:
#!unzip "/content/drive/MyDrive/data_science/petfinder-pawpularity-score.zip"

# Оптимизация гиперпараметров модели

Поиск оптимальных параметров осуществляется через фреймворк Optuna [1].

Преимущества использования Optuna:

- использует байесовские алгоритмы для оптимизации параметров;
- возможность ранней остановки неперспективного эксперимента исходя из статистики (например, если результаты эксперимента ниже медианы из уже рассмотренных);
- хорошая документация;
- возможность хранения результатов в базе данных, восстановление поиска гиперпараметров из базы данных;
- параллельный поиск параметров с разных устройств с помощью базы данных;
- один из самых быстрых фреймворков оптимизации гиперпараметров (в 1.5 раза быстрее hyperopt) [2] .


## Инициализация конфигурации модели

In [6]:
SEED = 2022

config = {'seed': SEED,
          'get_optim_params': True,
          'orig_train_csv': f'{INPUT_DATA_PREFIX}/train.csv',
          'orig_test_csv': f'{INPUT_DATA_PREFIX}/test.csv',
          'model': {
              'model_name': 'swin_base_patch4_window7_224_in22k', 
              'output_dims': [1022, 1], 
              'use_weights': False, 
              'plot_epoch_loss': True,
              'dropout': 0.3232967040113801, 
              'learning_rate': 4.812096281914617e-06, 
              'l2_regularization': 0.2766337962203344,
              'adam_betas': (0.37438095497909607, 0.7504122450294624),
              'pretrained': True,
              'seed': SEED,
              'full_trainable': True,
              'use_mixup': True,
              'mixup_alpha': 0.5, 
              'mixup_prob': 0.5,
          },
          'dataloader': {
              'seed': SEED,
              'n_splits': 5,
              'size_dataset': None,
              'use_kldiv': False,
              'train_photo_dir': f'{INPUT_DATA_PREFIX}/train/',
              'test_photo_dir': f'{INPUT_DATA_PREFIX}/test/',
              'train_csv': "files/mining_train.csv",
              'test_csv': "files/mining_test.csv",
              'dataset_params': {
                    'gaussian_sigma': 2,
                    'pca_for_add_features': False,
                    'std_for_add_features': True,
                    'p_vflip': 0.5,
                    'p_hflip': 0.5,
                    'val_augmentation': False,
                    'train_augmentation': True,
                    'image_size': 224,
                    'class_weights': None,
                    'seed': SEED,
               },
              'train_loader_params': {
                    'batch_size': 32,
                    'shuffle': True,
                    'num_workers': 2,
                    'pin_memory': False,
                    'drop_last': True,
              },
              'val_loader_params': {
                    'batch_size': 32,
                    'shuffle': False,
                    'num_workers': 2,
                    'pin_memory': False,
                    'drop_last': False
              },
              'test_loader_params': {
                    'batch_size': 1,
                    'shuffle': False,
                    'num_workers': 2,
                    'pin_memory': False,
                    'drop_last': False
              }
          },
         'trainer': {
              'max_epochs': 20,
              'gpus': 1 if torch.cuda.is_available() else 0,
              'progress_bar_refresh_rate': 10,
              'resume_from_checkpoint': None,
          },
}

Данные для обучение читаем из файла files/mining_train.csv, созданный в ноутбуке майнингу данных. Но тестовые требуется сгенерировать с помощью PetDataMining из оригинального датасета для тестирования в закрытой части соревнования. Так как этот датафрейм используется в классе PetDataLoader.

In [7]:
data_miner = PetDataMining(petfinder_csv=config['orig_test_csv'], drop_duplicates=False, plot_duplicate=False,
                 duplicate_thresh=0.9, plot_detector=False,
                 image_filepath=config['dataloader']['test_photo_dir'])
test_mining_df = data_miner.start()
test_mining_df.to_csv(config['dataloader']['test_csv'], index=False)

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-1-20 torch 1.10.0+cu111 CPU

Fusing layers... 
Model Summary: 574 layers, 140730220 parameters, 0 gradients
Adding AutoShape... 
  0%|          | 0/8 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/autocast_mode.py:141: UserWarning:

User provided device_type of 'cuda', but CUDA is not available. Disabling

 12%|█▎        | 1/8 [00:05<00:36,  5.19s/it]/usr/local/lib/python3.7/dist-packages/torch/autocast_mode.py:141: UserWarning:

User provided device_type of 'cuda', but CUDA is not available. Disabling

 25%|██▌       | 2/8 [00:10<00:30,  5.16s/it]/usr/local/lib/python3.7/dist-packages/torch/autocast_mode.py:141: UserWarning:

User provided device_type of 'cuda', but CUDA is not available. Disabling

 38%|███▊      | 3/8 [00:15<00:25,  5.18s/it]/usr/local/lib/python3.7/dist-packages/torch/autocast_mode.py:141: UserWarning:

User provided device_type of 'cuda', but CUDA is not available. Di

Фунция оптимизации для Optuna

In [13]:
def objective(trial: optuna.trial.Trial, config=config, size_dataset=5000) -> float:
    """ Функция оптимизации гиперпараметров модели

    ВНИМАНИЕ: Функция изменяет словарь config.

    Оптимизируются следующие параметры:
    - количество слоев в полносвязной сети на выходе экстрактора признаков;
    - количество нейронов в каждом слое полносвязной сети;
    - значение dropout в полносвязной сети;
    - скорость обучения learning_rate;
    - регуляризация l2_regularization;
    - значения beta1 и beta2 для AdamW оптимизатора;
    - включение/отключение MixUp;
    - включение/отключение аугментации;
    - включение стандартизации дополнительных признаков std_for_add_features;
    - включение преобразования ЗСФ для дополнительных признаков pca_for_add_features;

    Параметры
    ---------
    trial : optuna.trial.Trial
    config : dict
      Словарь конфигурации модели
    size_dataset : int
      Размер датасета для обучения. По умолчанию 5000.
    """

    # количество слоев в полносвязной сети на выходе экстрактора признаков
    n_layers = trial.suggest_int("n_layers", 0, 5)

    # количество нейронов в каждом слое полносвязной сети;
    output_dims = [
        trial.suggest_int("n_units_l{}".format(i), 16, 1024, log=True) for i in range(n_layers)
    ]
    output_dims.append(1)

    # значение dropout в полносвязной сети;
    dropout = trial.suggest_float("dropout", 0.1, 0.7)
    
    # Можно подобрать тип модели, но выберем его константой
    #model_name = trial.suggest_categorical('model_name', ['vgg16', 'resnet18', 'googlenet', 'alexnet'])
    model_name = 'swin_tiny_patch4_window7_224'
    
    # скорость обучения learning_rate
    learning_rate = trial.suggest_loguniform('learning_rate',  1e-6, 0.01)

    # регуляризация l2_regularization
    l2_regularization = trial.suggest_float('l2_regularization', 1e-6, 1.0)

    # значения beta1 и beta2 для AdamW оптимизатора
    adam_betas = (trial.suggest_float('adam_b1', 0.1, 0.999), trial.suggest_float('adam_b2', 0.1, 0.999))

    # включение/отключение MixUp
    use_mixup = trial.suggest_categorical('use_mixup', [True, False])

    # включение стандартизации дополнительных признаков std_for_add_features
    std_for_add_features = trial.suggest_categorical('std_for_add_features', [True, False])
    config['dataloader']['dataset_params']['std_for_add_features'] = std_for_add_features
    # включение преобразования ЗСФ для дополнительных признаков pca_for_add_features
    pca_for_add_features = trial.suggest_categorical('pca_for_add_features', [True, False])
    config['dataloader']['dataset_params']['pca_for_add_features'] = pca_for_add_features

    plot_epoch_loss = False
    full_trainable = True

    use_weights = config['model']['use_weights']
    mixup_alpha = config['model']['mixup_alpha']
    mixup_prob = config['model']['mixup_prob']

    model = PetFinderTransferModelBCEWithAddFeatures(model_name=model_name, dropout=dropout, output_dims=output_dims,
                                                     learning_rate=learning_rate, l2_regularization=l2_regularization,
                                                     adam_betas=adam_betas, use_weights=use_weights, plot_epoch_loss=plot_epoch_loss,
                                                     full_trainable=full_trainable, use_mixup=use_mixup, mixup_alpha=mixup_alpha,
                                                     mixup_prob=mixup_prob)

    config['dataloader']['size_dataset'] = size_dataset
    datamodule = PetDataLoader.create_kfold_loaders(**config['dataloader'])[0]

    callbacks = [PyTorchLightningPruningCallback(trial, monitor="val_rmse")]

    trainer = pl.Trainer(
        logger=True,
        log_every_n_steps=10,
        checkpoint_callback=False,
        max_epochs=5,
        gpus=1 if torch.cuda.is_available() else None,
        callbacks=callbacks,
    )

    hyperparameters = dict(model_name=model_name, n_layers=n_layers, dropout=dropout, output_dims=output_dims, 
                           std_for_add_features=std_for_add_features, pca_for_add_features=pca_for_add_features,
                           l2_regularization=l2_regularization, adam_b1=adam_betas[0], adam_beta2=adam_betas[1],
                           learning_rate=learning_rate, use_weights=use_weights, full_trainable=full_trainable,
                           use_mixup=use_mixup, mixup_alpha=mixup_alpha, mixup_prob=mixup_prob)
    
    trainer.logger.log_hyperparams(hyperparameters)
    trainer.fit(model, datamodule=datamodule)

    if len(model.train_history['val_rmse']) > 0:
        best_val_rmse = np.min(model.train_history['val_rmse'])
    else:
        best_val_rmse = 99999

    epoch_counts = len(model.train_history['val_rmse'])

    return best_val_rmse

def print_best_trial(study):
    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Number: {}".format(trial.number))
    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

## Цикл оптимизации гиперпараметров

Создаем обучение с помощью optuna.create_study(). Указываем имя и путь к базе данных. А также указываем в параметре direction, что нам необходимо минимизировать функцию.

In [ ]:
if config['get_optim_params']:
    
    pruner = optuna.pruners.MedianPruner()

    study = optuna.create_study(study_name="swin_tiny_patch4_window7_224", direction="minimize", 
                                pruner=pruner, load_if_exists=True,
                                storage="sqlite:////optuna.db")

    if len(study.best_trials) > 0:
        print_best_trial(study)

    study.optimize(objective, n_trials=1000)

    print_best_trial(study)